<a href="https://colab.research.google.com/github/PondKann/CXR-Project/blob/main/TransferL_KaggleDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
train_dir = '/content/drive/My Drive/Senior Project/xray_dataset_covid19/train'
validation_dir = '/content/drive/My Drive/Senior Project/xray_dataset_covid19/test'

In [6]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [7]:
train_generator = data_gen_train.flow_from_directory(train_dir,target_size=(128,128), batch_size=128, class_mode= "binary" )
valid_generator = data_gen_train.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode= "binary" )

Found 148 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


### MobileNetV2

In [8]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(128,128,3), include_top=False, weights="imagenet")

9420800/9406464 [==============================] - 0s 0us/step


In [9]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

In [11]:
base_model.trainable=False

Add custom head

In [12]:
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
prediction_layer = tf.keras.layers.Dense(units=1, activation="sigmoid")(average_pooling_layer)

In [14]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']           

Train model

In [16]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [18]:
model.fit_generator(train_generator, epochs=10, validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/10
2/2 [==============================] - 9s 8s/step - loss: 1.0361 - accuracy: 0.4054 - val_loss: 1.2991 - val_accuracy: 0.2750
Epoch 2/10
2/2 [==============================] - 8s 2s/step - loss: 1.0186 - accuracy: 0.4054 - val_loss: 1.2699 - val_accuracy: 0.2750
Epoch 3/10
2/2 [==============================] - 8s 7s/step - loss: 0.9922 - accuracy: 0.4122 - val_loss: 1.2437 - val_accuracy: 0.2750
Epoch 4/10
2/2 [==============================] - 8s 2s/step - loss: 0.9746 - accuracy: 0.4122 - val_loss: 1.2150 - val_accuracy: 0.2750
Epoch 5/10
2/2 [==============================] - 8s 2s/step - loss: 0.9495 - accuracy: 0.3919 - val_loss: 1.1888 - val_accuracy: 0.2750
Epoch 6/10
2/2 [==============================] - 7s 2s/step - loss: 0.9275 - accuracy: 0.3851 - val_loss: 1.1630 - val_accuracy: 0.2500
Epoch 7/10
2/2 [==============================] - 8s 7s/step - loss: 0.9018 - accuracy: 0.3851 - val_loss: 1.1396 - val_accuracy: 0.2500
Epoch 8/10
2/2 [=========================

Fine tuning

In [19]:
len(base_model.layers)

154

In [20]:
base_model.trainable = True

In [21]:
for layer in base_model.layers[:100]:    #fix w & bias in layer 0-100 
    layer.trainable = False

In [28]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [29]:
model.fit_generator(train_generator, epochs=10, 
                    validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10
2/2 [==============================] - 16s 3s/step - loss: 0.7160 - accuracy: 0.5338 - val_loss: 0.4612 - val_accuracy: 0.7500
Epoch 2/10
2/2 [==============================] - 9s 2s/step - loss: 0.0516 - accuracy: 1.0000 - val_loss: 0.4404 - val_accuracy: 0.7750
Epoch 3/10
2/2 [==============================] - 9s 2s/step - loss: 0.0242 - accuracy: 1.0000 - val_loss: 0.3926 - val_accuracy: 0.8000
Epoch 4/10
2/2 [==============================] - 9s 8s/step - loss: 0.0164 - accuracy: 1.0000 - val_loss: 0.4069 - val_accuracy: 0.8000
Epoch 5/10
2/2 [==============================] - 9s 2s/step - loss: 0.0137 - accuracy: 1.0000 - val_loss: 0.3814 - val_accuracy: 0.8250
Epoch 6/10
2/2 [==============================] - 9s 2s/step - loss: 0.0101 - accuracy: 1.0000 - val_loss: 0.3829 - val_accuracy: 0.8250
Epoch 7/10
2/2 [==============================] - 9s 8s/step - loss: 0.0072 - accuracy: 1.0000 - val_loss: 0.3499 - val_accuracy: 0.8250
Epoch 8/10
2/2 [========================

### DenseNet201

In [36]:
base_model = tf.keras.applications.DenseNet201(input_shape=(128,128,3), include_top=False, weights="imagenet")

In [37]:
base_model.summary()

Model: "densenet201"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_4 (ZeroPadding2  (None, 134, 134, 3)  0          ['input_4[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 64, 64, 64)   9408        ['zero_padding2d_4[0][0]']       
                                                                                        

In [38]:
base_model.trainable=False

Add custom head

In [39]:
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
prediction_layer = tf.keras.layers.Dense(units=1, activation="sigmoid")(average_pooling_layer)

In [40]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [41]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_4 (ZeroPadding2  (None, 134, 134, 3)  0          ['input_4[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 64, 64, 64)   9408        ['zero_padding2d_4[0][0]']       
                                                                                            

Train model

In [42]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [43]:
model.fit_generator(train_generator, epochs=10, validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/10
2/2 [==============================] - 33s 19s/step - loss: 0.6518 - accuracy: 0.5878 - val_loss: 0.6497 - val_accuracy: 0.6250
Epoch 2/10
2/2 [==============================] - 18s 15s/step - loss: 0.5669 - accuracy: 0.6284 - val_loss: 0.5841 - val_accuracy: 0.6500
Epoch 3/10
2/2 [==============================] - 19s 6s/step - loss: 0.5333 - accuracy: 0.6689 - val_loss: 0.5400 - val_accuracy: 0.7000
Epoch 4/10
2/2 [==============================] - 19s 16s/step - loss: 0.4924 - accuracy: 0.7365 - val_loss: 0.5146 - val_accuracy: 0.7000
Epoch 5/10
2/2 [==============================] - 18s 16s/step - loss: 0.4646 - accuracy: 0.7770 - val_loss: 0.4864 - val_accuracy: 0.7250
Epoch 6/10
2/2 [==============================] - 19s 6s/step - loss: 0.4486 - accuracy: 0.8041 - val_loss: 0.4614 - val_accuracy: 0.7500
Epoch 7/10
2/2 [==============================] - 19s 6s/step - loss: 0.4268 - accuracy: 0.8176 - val_loss: 0.4372 - val_accuracy: 0.7250
Epoch 8/10
2/2 [==============

Fine tuning

In [44]:
len(base_model.layers)

707

In [45]:
base_model.trainable = True

In [46]:
for layer in base_model.layers[:500]:    #fix w & bias in layer 0-500
    layer.trainable = False

In [47]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [48]:
model.fit_generator(train_generator, epochs=10, 
                    validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10
2/2 [==============================] - 44s 22s/step - loss: 0.2348 - accuracy: 0.9324 - val_loss: 0.0912 - val_accuracy: 0.9750
Epoch 2/10
2/2 [==============================] - 21s 17s/step - loss: 0.0976 - accuracy: 0.9730 - val_loss: 0.0746 - val_accuracy: 0.9750
Epoch 3/10
2/2 [==============================] - 21s 5s/step - loss: 0.0209 - accuracy: 1.0000 - val_loss: 0.0719 - val_accuracy: 0.9750
Epoch 4/10
2/2 [==============================] - 21s 18s/step - loss: 0.0136 - accuracy: 1.0000 - val_loss: 0.0733 - val_accuracy: 0.9750
Epoch 5/10
2/2 [==============================] - 21s 17s/step - loss: 0.0106 - accuracy: 1.0000 - val_loss: 0.0698 - val_accuracy: 0.9750
Epoch 6/10
2/2 [==============================] - 21s 5s/step - loss: 0.0080 - accuracy: 1.0000 - val_loss: 0.0669 - val_accuracy: 0.9750
Epoch 7/10
2/2 [==============================] - 21s 17s/step - loss: 0.0071 - accuracy: 1.0000 - val_loss: 0.0650 - val_accuracy: 0.9750
Epoch 8/10
2/2 [=============

In [ ]:
tf.keras.applications.DenseNet201(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
)

In [31]:
base_model2 = tf.keras.applications.DenseNet201(input_shape=(128,128,3), include_top=False, weights="imagenet")

74850304/74836368 [==============================] - 1s 0us/step


In [32]:
base_model2.summary()

Model: "densenet201"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 134, 134, 3)  0          ['input_3[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 64, 64, 64)   9408        ['zero_padding2d_2[0][0]']       
                                                                                        

In [33]:
base_model2.trainable=False

Add custom head

In [34]:
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model2.output)
prediction_layer = tf.keras.layers.Dense(units=1, activation="sigmoid")(average_pooling_layer)

In [35]:
model2 = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

ValueError: ignored

In [ ]:
model2.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']           

Train model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
model.fit_generator(train_generator, epochs=10, validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/10
2/2 [==============================] - 9s 8s/step - loss: 1.0361 - accuracy: 0.4054 - val_loss: 1.2991 - val_accuracy: 0.2750
Epoch 2/10
2/2 [==============================] - 8s 2s/step - loss: 1.0186 - accuracy: 0.4054 - val_loss: 1.2699 - val_accuracy: 0.2750
Epoch 3/10
2/2 [==============================] - 8s 7s/step - loss: 0.9922 - accuracy: 0.4122 - val_loss: 1.2437 - val_accuracy: 0.2750
Epoch 4/10
2/2 [==============================] - 8s 2s/step - loss: 0.9746 - accuracy: 0.4122 - val_loss: 1.2150 - val_accuracy: 0.2750
Epoch 5/10
2/2 [==============================] - 8s 2s/step - loss: 0.9495 - accuracy: 0.3919 - val_loss: 1.1888 - val_accuracy: 0.2750
Epoch 6/10
2/2 [==============================] - 7s 2s/step - loss: 0.9275 - accuracy: 0.3851 - val_loss: 1.1630 - val_accuracy: 0.2500
Epoch 7/10
2/2 [==============================] - 8s 7s/step - loss: 0.9018 - accuracy: 0.3851 - val_loss: 1.1396 - val_accuracy: 0.2500
Epoch 8/10
2/2 [=========================

Fine tuning

In [ ]:
len(base_model.layers)

154

In [ ]:
base_model.trainable = True

In [ ]:
for layer in base_model.layers[:100]:
    layer.trainable = False

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
model.fit_generator(train_generator, epochs=10, 
                    validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10
2/2 [==============================] - 16s 3s/step - loss: 0.7160 - accuracy: 0.5338 - val_loss: 0.4612 - val_accuracy: 0.7500
Epoch 2/10
2/2 [==============================] - 9s 2s/step - loss: 0.0516 - accuracy: 1.0000 - val_loss: 0.4404 - val_accuracy: 0.7750
Epoch 3/10
2/2 [==============================] - 9s 2s/step - loss: 0.0242 - accuracy: 1.0000 - val_loss: 0.3926 - val_accuracy: 0.8000
Epoch 4/10
2/2 [==============================] - 9s 8s/step - loss: 0.0164 - accuracy: 1.0000 - val_loss: 0.4069 - val_accuracy: 0.8000
Epoch 5/10
2/2 [==============================] - 9s 2s/step - loss: 0.0137 - accuracy: 1.0000 - val_loss: 0.3814 - val_accuracy: 0.8250
Epoch 6/10
2/2 [==============================] - 9s 2s/step - loss: 0.0101 - accuracy: 1.0000 - val_loss: 0.3829 - val_accuracy: 0.8250
Epoch 7/10
2/2 [==============================] - 9s 8s/step - loss: 0.0072 - accuracy: 1.0000 - val_loss: 0.3499 - val_accuracy: 0.8250
Epoch 8/10
2/2 [========================